In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install beautifulsoup4
!pip install lxml
!pip install html5lib
!pip install requests

from bs4 import BeautifulSoup

import requests
import urllib.request
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.21.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [25]:
#source =  requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').json()

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#download the URL and extract the content to the variable html 
request = urllib.request.Request(url)
html = urllib.request.urlopen(request).read()

soup = BeautifulSoup(html, "html.parser")

site_table = soup.table
column_names=[]
for data in site_table.find_all('th'):
    column_names.append(data.text.strip())
    

column_names

['Postcode', 'Borough', 'Neighbourhood']

In [5]:
info_data = []
for info in site_table.find_all('tr'):
    info_data.append(info.text.strip())

neighbourhood = []
list_tor = []

for i in range(len(info_data)):
    x = info_data[i].split('\n')
    list_tor.append(x)
    
for j in range(len(list_tor)-1):
    post = list_tor[j+1][0]
    bor = list_tor[j+1][1]
    nei = list_tor[j+1][2]
    neighbourhood.append({'Postcode': post, 'Borough': bor, 'Neighbourhood': nei})

neighbourhood = pd.DataFrame.from_dict(neighbourhood)
neighbourhood.head()

,Borough,Neighbourhood,Postcode
0,Not assigned,Not assigned,M1A
1,Not assigned,Not assigned,M2A
2,North York,Parkwoods,M3A
3,North York,Victoria Village,M4A
4,Downtown Toronto,Harbourfront,M5A


In [18]:
arranged_cols = [neighbourhood.columns[-1]] + [neighbourhood.columns[0]] + [neighbourhood.columns[1]]
#arranged_cols
df_toronto = neighbourhood[arranged_cols]
df_toronto.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [24]:
df_tor = df_toronto[df_toronto['Borough']!= 'Not assigned'].reset_index()
df_tor.drop('index', axis=1, inplace=True)
df_tor.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
